## **Installing packages**

In [1]:
!pip3 install torch torchvision torchaudio torchsummary
!pip install 'tqdm'

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [2]:
!pip install tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 1.1 MB/s eta 0:00:00


In [3]:
!pip install -U fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 868.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=705334774a1a7cf03815186869d35c21349f1ab9b79ca914d6701e3e75dad43b
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=41e2bc8d7f7cfa475ea186f1a1ff9ed6d3fcc9b22d76d06e452c71109882d042
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath


## **Libraries**

In [4]:
import os
import os.path
import sys
import logging

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader, Dataset
from torch.backends import cudnn
import torch.cuda.amp as amp
import torch.cpu.amp as amp_cpu
from torch.autograd import Function
import torch.nn.functional as F

import torchvision
from torchvision.transforms import transforms

from PIL import Image
from tqdm import tqdm
import numpy as np
from tensorboardX import SummaryWriter

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import random

from pathlib import Path
import json
import random

from torchsummary import summary
from fvcore.nn import FlopCountAnalysis


logger = logging.getLogger()

## **Import**

### Importing datasets

In [ ]:
from google.colab import drive

drive.mount('/content/Drive')

cityscapes = True
gta5 = True

if not os.path.isdir(f'/content/Cityscapes') and cityscapes:
  !jar xvf  "/content/Drive/MyDrive/Colab Notebooks/AML/Cityscapes.zip"
if not os.path.isdir(f'/content/GTA5') and gta5:
  !jar xvf  "/content/Drive/MyDrive/Colab Notebooks/AML/GTA5.zip"

### Import methods from TA's repository

In [6]:
# cloning github repo for model (BiseNet with STDC) and utils, I rewrote manually the Train.py and Cityscapes.py below
import pathlib
print(pathlib.Path.cwd())
!git clone https://github.com/ClaudiaCuttano/AML_Semantic_DA.git


# importing stuff from the repo we just cloned

# copied from train.py
from AML_Semantic_DA.model.model_stages import BiSeNet  # see https://github.com/ClaudiaCuttano/AML_Semantic_DA/blob/master/model/model_stages.py
from AML_Semantic_DA.utils import poly_lr_scheduler, reverse_one_hot, compute_global_accuracy, fast_hist, per_class_iu # see https://github.com/ClaudiaCuttano/AML_Semantic_DA/blob/master/utils.py

/content
Cloning into 'AML_Semantic_DA'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 18 (delta 0), reused 0 (delta 0), pack-reused 9
Receiving objects: 100% (18/18), 10.88 KiB | 5.44 MiB/s, done.


### Import methods from our repository

In [21]:
# cloning github repo for model (BiseNet with STDC) and utils, I rewrote manually the Train.py and Cityscapes.py below
import pathlib
print(pathlib.Path.cwd())
!git clone https://github.com/ivanmag22/Semantic_Segmentation_project.git


# importing stuff from the repo we just cloned
from Semantic_Segmentation_project.model.model_stages import BiSeNet
from Semantic_Segmentation_project.model.bisenetv1 import BiSeNet as BiSeNetv1

/content
fatal: destination path 'Semantic_Segmentation_project' already exists and is not an empty directory.


### Import methods for GTA5
Clone repo useful to give a label for each pixel.

In [8]:
import pathlib
print(pathlib.Path.cwd())
!git clone https://github.com/MichaelFan01/STDC-Seg.git STDC_seg
# importing stuff from the repo we just cloned

/content
Cloning into 'STDC_seg'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 98 (delta 18), reused 8 (delta 8), pack-reused 70
Receiving objects: 100% (98/98), 1.93 MiB | 7.27 MiB/s, done.
Resolving deltas: 100% (24/24), done.


## **Data loader classes + Preprocessing**

### Data Pre Processing
We need to do pre-processing only on **training set** (and not on validation set because on the last one we work with 1024 x 2048 pictures)

In [9]:
train_transform = transforms.Compose([transforms.Resize((512,1024), transforms.InterpolationMode.BILINEAR),
                                      transforms.ToTensor()
                                      ])
label_transform = transforms.Resize((512,1024), transforms.InterpolationMode.NEAREST)
eval_transform = transforms.Compose([transforms.ToTensor()])


gta_train_transform = transforms.Compose([transforms.Resize((512,1024), transforms.InterpolationMode.BILINEAR),
                                          transforms.ToTensor()
                                          ])
gta_label_transform = transforms.Resize((512,1024), transforms.InterpolationMode.NEAREST)
gta_val_transform = transforms.Compose([transforms.ToTensor()])


#data augmentation
bright_t = transforms.ColorJitter(brightness=[1,2])
contrast_t = transforms.ColorJitter(contrast = [2,5])
saturation_t = transforms.ColorJitter(saturation = [1,3])
hue_t = transforms.ColorJitter(hue = 0.2)
gs_t = transforms.Grayscale(3)
hflip_t = transforms.RandomHorizontalFlip(p = 1)
cc_t = transforms.CenterCrop((256,512))

augmentation_transforms = [bright_t, contrast_t, saturation_t, hue_t, gs_t, hflip_t, cc_t]

### Cityscapes

In [10]:
class CityScapes(Dataset):
    def __init__(self, base_root, mode):
        super(CityScapes, self).__init__()

        self.mode = mode
        self.image_paths = [] # images
        self.mask_paths_colored = [] # colored images
        self.mask_paths_bw = [] # labels

        assert(mode == 'train' or mode == 'val')  # just checking for potential issues
        image_folder = f'{base_root}images/{mode}'

        for root, dirs, files in os.walk(image_folder):
            for file_name in files:
                image_path = f'{root}/{file_name}'
                assert(Path(image_path).is_file())
                self.image_paths.append(image_path)

                mask_path_bw = image_path.replace('leftImg8bit', 'gtFine_labelTrainIds')
                mask_path_bw = mask_path_bw.replace('/images/', '/gtFine/')
                assert(Path(mask_path_bw).is_file())
                self.mask_paths_bw.append(mask_path_bw)

        if self.mode == 'train':
            self.image_transform = train_transform
        if self.mode == 'val':
            self.image_transform = eval_transform
        self.label_transform = label_transform

        assert (len(self.image_paths) != 0)
        assert (len(self.image_paths) == len(self.mask_paths_bw))


    def __getitem__(self, index):

        image = Image.open(self.image_paths[index]).convert('RGB')
        label = Image.open(self.mask_paths_bw[index])

        if self.mode == 'train':
            label = np.array(self.label_transform(label))[np.newaxis, :]
        else:
            label = np.array(label)[np.newaxis, :]

        image = self.image_transform(image)

        return image, label


    def __len__(self):
        return len(self.image_paths)

### GTA5

In [11]:
class GTA5(Dataset):
    def __init__(self, base_root, mode, augmentation=False, train_test_rateo=2/3):
        super(GTA5, self).__init__()

        self.mode = mode
        self.image_paths = [] # images
        self.label_paths = [] # labels
        self.train_test_rateo = train_test_rateo
        self.augmentation = augmentation
        with open('STDC_seg/cityscapes_info.json', 'r') as fr:
            labels_info = json.load(fr)
        self.label_map = {el['id']: el['trainId'] for el in labels_info}
        self.label_map.update({34 : 255})

        assert(mode == 'train' or mode == 'val')  # just checking for potential issues

        image_folder = f'{base_root}images'

        for root, dirs, files in os.walk(image_folder):
            for file_name in files:
                image_path = f'{root}/{file_name}'
                assert(Path(image_path).is_file())
                self.image_paths.append(image_path)

                label_path = image_path.replace('/images/', '/labels/')
                assert(Path(label_path).is_file())
                self.label_paths.append(label_path)

        l = int(len(self.image_paths) * self.train_test_rateo)
        if self.mode == 'train':
            self.image_paths = self.image_paths[:l]
            self.label_paths = self.label_paths[:l]
            self.image_transform = gta_train_transform
            self.label_transform = gta_label_transform
        elif self.mode == 'val':
            self.image_paths = self.image_paths[l:]
            self.label_paths = self.label_paths[l:]
            self.image_transform = gta_val_transform

        assert (len(self.image_paths) != 0)
        assert (len(self.image_paths) == len(self.label_paths))


    def __getitem__(self, index):

        image = Image.open(self.image_paths[index]).convert('RGB')
        image = self.image_transform(image)

        label = Image.open(self.label_paths[index])
        if self.mode == 'train':
            label = self.label_transform(label)

        if self.augmentation and random.choice([True, False]) and self.mode == 'train':
            idx = random.randint(0, 6)

            image = augmentation_transforms[idx](image)

            if hflip_t is augmentation_transforms[idx] or cc_t is augmentation_transforms[idx]:
                label = augmentation_transforms[idx](label)

            if cc_t is augmentation_transforms[idx]:
                rimage_t = transforms.Resize((512,1024), transforms.InterpolationMode.BILINEAR, antialias=None)
                rlabel_t = transforms.Resize((512,1024), transforms.InterpolationMode.NEAREST, antialias=None)
                image = rimage_t(image)
                label = augmentation_transforms[idx](label)
                label = rlabel_t(label)

        label = np.array(label).astype(np.int64)[np.newaxis, :]
        label = self.convert_labels(label)

        return image, label


    def convert_labels(self, label):
        for k, v in self.label_map.items():
            label[label == k] = v
        return label


    def __len__(self):
        return len(self.image_paths)


## **Architecture**
BiSeNet as Domain Adaptation Neural Network (DANN)

In [12]:
class DepthWiseSeparableConvolution(nn.Module):
    def __init__(self, ch_in, ch_out):
        super(DepthWiseSeparableConvolution, self).__init__()
        self.depth_wise = nn.Conv2d(ch_in, ch_in, kernel_size=4, stride=2, padding=1, groups=ch_in)
        self.point_wise = nn.Conv2d(ch_in, ch_out, kernel_size=1)

    def forward(self, x):
        out = self.depth_wise(x)
        out = self.point_wise(out)
        return out

class LightFCDiscriminator(nn.Module):
    def __init__(self, num_classes, ndf=64):
        super(LightFCDiscriminator, self).__init__()

        # context
        self.conv1 = DepthWiseSeparableConvolution(num_classes, ndf)
        self.conv2 = DepthWiseSeparableConvolution(ndf, ndf*2)
        self.conv3 = DepthWiseSeparableConvolution(ndf*2, ndf*4)
        self.conv4 = DepthWiseSeparableConvolution(ndf*4, ndf*8)
        self.classifier = DepthWiseSeparableConvolution(ndf*8, 1)
        self.leaky_relu = nn.LeakyReLU(negative_slope=0.2, inplace=True)
        self.up_sample = nn.Upsample(scale_factor=32, mode='bilinear')


    def forward(self, x):
        x = self.conv1(x)
        x = self.leaky_relu(x)
        x = self.conv2(x)
        x = self.leaky_relu(x)
        x = self.conv3(x)
        x = self.leaky_relu(x)
        x = self.conv4(x)
        x = self.leaky_relu(x)
        x = self.classifier(x)
        x = self.up_sample(x)
        return x

class LightLightFCDiscriminator(nn.Module):
    def __init__(self, num_classes, ndf=64):
        super(LightLightFCDiscriminator, self).__init__()

        # context
        self.conv1 = DepthWiseSeparableConvolution(num_classes, ndf)
        self.conv2 = DepthWiseSeparableConvolution(ndf, ndf*2)
        self.classifier = DepthWiseSeparableConvolution(ndf*2, 1)
        self.leaky_relu = nn.LeakyReLU(negative_slope=0.2, inplace=True)
        self.up_sample = nn.Upsample(scale_factor=8, mode='bilinear')


    def forward(self, x):
        x = self.conv1(x)
        x = self.leaky_relu(x)
        x = self.conv2(x)
        x = self.leaky_relu(x)
        x = self.classifier(x)
        x = self.up_sample(x)
        return x

class FCDiscriminator(nn.Module):

    def __init__(self, num_classes, ndf = 64):
        super(FCDiscriminator, self).__init__()

        self.conv1 = nn.Conv2d(num_classes, ndf, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.Conv2d(ndf, ndf*2, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.Conv2d(ndf*2, ndf*4, kernel_size=4, stride=2, padding=1)
        self.conv4 = nn.Conv2d(ndf*4, ndf*8, kernel_size=4, stride=2, padding=1)
        self.classifier = nn.Conv2d(ndf*8, 1, kernel_size=4, stride=2, padding=1)

        self.leaky_relu = nn.LeakyReLU(negative_slope=0.2, inplace=True)
        self.up_sample = nn.Upsample(scale_factor=32, mode='bilinear')
        #self.sigmoid = nn.Sigmoid()


    def forward(self, x):
        x = self.conv1(x)
        x = self.leaky_relu(x)
        x = self.conv2(x)
        x = self.leaky_relu(x)
        x = self.conv3(x)
        x = self.leaky_relu(x)
        x = self.conv4(x)
        x = self.leaky_relu(x)
        x = self.classifier(x)
        x = self.up_sample(x)
        #x = self.sigmoid(x)

        return x

## **Train + Validation**

### Base Training

In [13]:
from torch.autograd import Variable

def train(args, model, optimizer, dataloader_train, dataloader_val):
    print("start train without domain adaptation")
    # for SummaryWriter read (https://pytorch.org/docs/stable/tensorboard.html)
    writer = SummaryWriter(log_dir='/content/Drive/MyDrive/AML project/logs', comment=''.format(args.optimizer)) # log is in run/ folder

    loss_func = torch.nn.CrossEntropyLoss(ignore_index=255)
    max_miou = 0
    step = 0

    for epoch in range(args.epoch_start_i+1, args.num_epochs+1):
        lr = poly_lr_scheduler(optimizer, args.learning_rate, iter=epoch, max_iter=args.num_epochs)
        model.train() # Sets module in training mode
        tq = tqdm(total=len(dataloader_train) * args.batch_size)
        tq.set_description('epoch %d, lr %f' % (epoch, lr))
        loss_record = []

        for i, (data, label) in enumerate(dataloader_train):
            label = label.long()

            optimizer.zero_grad() # Zero-ing the gradients


            output, out16, out32 = model(data)  # Forward pass to the network
            # Compute loss based on output and ground truth
            loss1 = loss_func(output, label.squeeze(1))
            loss2 = loss_func(out16, label.squeeze(1))
            loss3 = loss_func(out32, label.squeeze(1))
            loss = loss1 + loss2 + loss3  # sum of losses

            loss.backward()
            optimizer.step()

            tq.update(args.batch_size)
            tq.set_postfix(loss='%.6f' % loss)
            step += 1
            writer.add_scalar('loss_step', loss, step)
            loss_record.append(loss.item())
        tq.close()
        loss_train_mean = np.mean(loss_record)
        writer.add_scalar('epoch/loss_epoch_train', float(loss_train_mean), epoch)
        print('loss for train : %f' % (loss_train_mean))

        if epoch % args.checkpoint_step == 0 and epoch != 0:
            import os
            if not os.path.isdir(args.save_model_path):
                os.mkdir(args.save_model_path)
            torch.save(model.module.state_dict(), f'{args.save_model_path}Saved_model_epoch_{epoch}.pth')

        if epoch % args.validation_step == 0 and epoch != args.num_epochs:
            precision, miou = val(args, model, dataloader_val)  # val() function call
            if miou > max_miou:
                max_miou = miou
                import os
                os.makedirs(args.save_model_path, exist_ok=True)
                torch.save(model.module.state_dict(), f'{args.save_model_path}Best_model_epoch_{epoch}.pth')
            writer.add_scalar('epoch/precision_val', precision, epoch)
            writer.add_scalar('epoch/miou val', miou, epoch)

### Domain Adaptation Training

In [14]:
from torch.autograd import Variable

def train_da(args, model, optimizer, model_D, optimizer_D, dataloader_train, dataloader_val, domain_adapt=False, dataloader_target=None):
    print("start train with domain adaptation")
    # for SummaryWriter read (https://pytorch.org/docs/stable/tensorboard.html)
    writer = SummaryWriter(log_dir='/content/Drive/MyDrive/AML project/logs', comment=''.format(args.optimizer)) # log is in run/ folder

    loss_func_G = torch.nn.CrossEntropyLoss(ignore_index=255)
    loss_func_adv = torch.nn.BCEWithLogitsLoss()
    loss_func_D = torch.nn.BCEWithLogitsLoss()

    max_miou = 0
    step = 0

    # see (https://www.github.com/wasidennis/AdaptSegNet/blob/master/train_gta2cityscapes_multi)
    print("Train DA")

    LAMBDA_ADV_TARGET = args.adv_factor

    dataloader_len = min(len(dataloader_train), len(dataloader_target))
    for epoch in range(args.epoch_start_i+1, args.num_epochs+1):
        lr = poly_lr_scheduler(optimizer, args.learning_rate, iter=epoch-1, max_iter=args.num_epochs)
        lr_D = poly_lr_scheduler(optimizer_D, args.learning_rate_D, iter=epoch-1, max_iter=args.num_epochs)

        model.train()
        model_D.train()

        tq = tqdm(total=dataloader_len * args.batch_size)

        tq.set_description('epoch %d, lr %f, lr_discriminator %f' % (epoch, lr, lr_D))

        # set the ground truth for the discriminator
        source_label = 0
        target_label = 1
        # initiate lists to track the losses
        loss_G_record = []                                                       # track the Segmentation loss
        loss_adv_record = []                                                     # track the advarsirial loss
        loss_D_record = []                                                       # track the discriminator loss

        source_train_loader = enumerate(dataloader_train)
        s_size = len(dataloader_train)
        target_loader = enumerate(dataloader_target)
        t_size = len(dataloader_target)

        for i in range(dataloader_len):

            optimizer.zero_grad()
            optimizer_D.zero_grad()

            # =====================================
            # train Generator G:
            # =====================================

            for param in model_D.parameters():
                param.requires_grad = False

            # Train with source:
            # =================================

            _, batch = next(source_train_loader)
            data, label = batch
            label = label.long()

            output_s, out16, out32 = model(data)
            loss1 = loss_func_G(output_s, label.squeeze(1))
            loss2 = loss_func_G(out16, label.squeeze(1))
            loss3 = loss_func_G(out32, label.squeeze(1))
            loss_G = loss1 + loss2 + loss3

            loss_G.backward()

            # Train with target:
            # =================================

            _, batch = next(target_loader)

            data, _ = batch

            output_t, _, _ = model(data)
            D_out = model_D(F.softmax(output_t, dim=1))
            loss_adv = loss_func_adv(D_out, Variable(torch.FloatTensor(D_out.data.size()).fill_(source_label)))
            loss_adv = loss_adv * LAMBDA_ADV_TARGET

            loss_adv.backward()

            # =====================================
            # train Discriminator D:
            # =====================================

            for param in model_D.parameters():
                param.requires_grad = True

            # Train with source:
            # =================================

            output_s = output_s.detach()

            D_out = model_D(F.softmax(output_s, dim=1))                                                                   # we feed the discriminator with the output of the G-model
            loss_D = loss_func_D(D_out, Variable(torch.FloatTensor(D_out.data.size()).fill_(source_label)))

            loss_D.backward()

            # Train with target:
            # =================================

            output_t = output_t.detach()
            D_out = model_D(F.softmax(output_t, dim=1))  # we feed the discriminator with the output of the model
            loss_D = loss_func_D(D_out, Variable(torch.FloatTensor(D_out.data.size()).fill_(target_label)))

            tq.update(args.batch_size)
            losses = {"loss_seg" : '%.6f' %(loss_G.item())  , "loss_adv" : '%.6f' %(loss_adv.item()) , "loss_D" : '%.6f'%(loss_D.item()) } # add dictionary to print losses
            tq.set_postfix(losses)

            loss_G_record.append(loss_G.item())
            loss_adv_record.append(loss_adv.item())
            loss_D_record.append(loss_D.item())
            step += 1
            writer.add_scalar('loss_G_step', loss_G, step)  # track the segmentation loss
            writer.add_scalar('loss_adv_step', loss_adv, step)  # track the adversarial loss
            writer.add_scalar('loss_D_step', loss_D, step)  # track the discreminator loss
            optimizer.step()  # update the optimizer for genarator
            optimizer_D.step()  # update the optimizer for discriminator
        tq.close()

        loss_seg_record_mean = np.mean(loss_G_record)
        loss_adv_record_mean = np.mean(loss_adv_record)
        loss_D_record_mean = np.mean(loss_D_record)
        loss_mean = np.mean([loss_seg_record_mean, loss_adv_record_mean, loss_D_record_mean])
        writer.add_scalar('epoch/loss_epoch_train', float(loss_mean), epoch)
        print(f'loss for train :\n - Segmentation: {loss_seg_record_mean}\n - Adversarial: {loss_adv_record_mean}\n - Discriminator: {loss_D_record_mean}')

        if epoch % args.checkpoint_step == 0 and epoch != 0:
            import os
            if not os.path.isdir(args.save_model_path):
                os.mkdir(args.save_model_path)
            torch.save(model.module.state_dict(), f'{args.save_model_path}cpu_Saved_model_epoch_{epoch}.pth')

        if epoch % args.validation_step == 0 and epoch != args.num_epochs:
            precision, miou = val(args, model, dataloader_val)  # val() function call
            if miou > max_miou:
                max_miou = miou
                import os
                os.makedirs(args.save_model_path, exist_ok=True)
                torch.save(model.module.state_dict(), f'{args.save_model_path}cpu_Best_model_epoch_{epoch}.pth')
            writer.add_scalar('epoch/precision_val', precision, epoch)
            writer.add_scalar('epoch/miou val', miou, epoch)

### Validation

In [15]:
def val(args, model, dataloader):
    print('start val!')
    with torch.no_grad():
        model.eval()
        precision_record = []
        hist = np.zeros((args.num_classes, args.num_classes))
        tq = tqdm(total=len(dataloader))
        tq.set_description('Validation')
        for i, (data, label) in enumerate(dataloader):
            label = label.type(torch.LongTensor)
            label.long()

            # get RGB predict image
            if args.model == 'STDC-net':
                predict, _, _ = model(data)
            else:
                predict = model(data)

            predict = predict.squeeze(0)
            predict = reverse_one_hot(predict)
            predict = np.array(predict.cpu())

            # get RGB label image
            label = label.squeeze()
            label = np.array(label.cpu())

            # compute per pixel accuracy
            precision = compute_global_accuracy(predict, label)
            hist += fast_hist(label.flatten(), predict.flatten(), args.num_classes)

            # there is no need to transform the one-hot array to visual RGB array
            precision_record.append(precision)

            tq.update(1)

        tq.close()
        precision = np.mean(precision_record)
        miou_list = per_class_iu(hist)
        miou = np.mean(miou_list)
        print('\nprecision per pixel for test: %.3f' % precision)
        print('mIoU for validation: %.3f' % miou)
        print(f'mIoU per class: {miou_list}')

        return precision, miou

## **Main**
Prepare arguments, Datasets, Dataloaders, model, training and test

### main definition

In [16]:
def main(args, eval_only=False):

    n_classes = args.num_classes
    train_root = args.train_root
    val_root = args.val_root
    target_root = args.train_root
    domain_adapt = args.domain_adaptation

    # defining the training, validation (and target for domain adaptation) datasets and dataloaders
    if train_root == 'GTA5/':
        if train_root != val_root: # if we only use GTA5 for training, use the entire dataset and don't leave a portion for testing
            train_dataset = GTA5(train_root, 'train', args.data_augmentation, 1)
        else:
            train_dataset = GTA5(train_root, 'train', args.data_augmentation)
    else:
        train_dataset = CityScapes(train_root, 'train')

    if val_root == 'GTA5/':
        val_dataset = GTA5(val_root, 'val')
    else:
        val_dataset = CityScapes(val_root, 'val')

    if target_root == 'GTA5/':
        target_dataset = GTA5(target_root, 'train')
    else:
        target_dataset = CityScapes(target_root, 'train')

    dataloader_train = DataLoader(train_dataset,
                      batch_size=args.batch_size,
                      shuffle=True,
                      num_workers=args.num_workers,
                      pin_memory=False,
                      drop_last=True)

    dataloader_val = DataLoader(val_dataset,
                      batch_size=1,
                      shuffle=False,
                      num_workers=args.num_workers,
                      drop_last=False)

    dataloader_target = DataLoader(target_dataset,
                      batch_size=args.batch_size,
                      shuffle=True,
                      num_workers=args.num_workers,
                      pin_memory=False,
                      drop_last=True)

    device = torch.device("cpu")

    ## model
    if args.model == 'STDC-net':
        backbone='CatmodelSmall'
        model = BiSeNet(backbone=backbone, n_classes=n_classes, pretrain_model=args.pretrain_path, use_conv_last=args.use_conv_last, device=device)
    else:
        backbone='resnet18'
        model = BiSeNetv1(num_classes=n_classes, context_path=backbone)

    ### load a saved model from start epoch
    if args.epoch_start_i != 0:
        print(f'loading data from saved model {args.saved_model}')
        model.load_state_dict(torch.load(f'{args.save_model_path}{args.saved_model}'))

    ## optimizer
    # build optimizer
    if args.optimizer == 'rmsprop':
        optimizer = torch.optim.RMSprop(model.parameters(), args.learning_rate)
    elif args.optimizer == 'sgd':
        optimizer = torch.optim.SGD(model.parameters(), args.learning_rate, momentum=0.9, weight_decay=1e-4)
    elif args.optimizer == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), args.learning_rate)
    else:  # rmsprop
        print('not supported optimizer \n')
        return None

    if domain_adapt:
        # init Discriminator
        if args.discr == 'fc':
            model_D = FCDiscriminator(args.num_classes)
        elif args.discr == 'light':
            model_D = LightFCDiscriminator(args.num_classes)
        elif args.discr == 'light-thin':
            model_D = LightLightFCDiscriminator(args.num_classes)

        if args.optimizer_D == 'rmsprop':
            optimizer_D = torch.optim.RMSprop(model_D.parameters(), args.learning_rate_D)
        elif args.optimizer_D == 'sgd':
            optimizer_D = torch.optim.SGD(model_D.parameters(), args.learning_rate_D, momentum=0.9, weight_decay=1e-4)
        elif args.optimizer_D == 'adam':
            optimizer_D = torch.optim.Adam(model_D.parameters(), args.learning_rate_D, betas=(0.9, 0.99))
        else:  # rmsprop
            print('not supported optimizer \n')
            return None

    if not eval_only: #this is for when we only care about evaluating a saved model and not about training
        if domain_adapt:
            ## train loop for domain adaptation
            train_da(args, model, optimizer, model_D, optimizer_D, dataloader_train, dataloader_val, domain_adapt=domain_adapt, dataloader_target=dataloader_target)
        else:
            print("Train and validation on the same dataset (different partitions)" if args.train_root==args.val_root else "Train Domain Shift")
            ## normal train loop
            train(args, model, optimizer, dataloader_train, dataloader_val)
    # final test
    val(args, model, dataloader_val)

### main execution

**Tasks**:
- Networks: model
  - 'STDC-net' for *STDC-net*
  - 'BiSeNetv1' for *BiSeNet v1* with ResNet-18 as backbone
1. train on Cityscapes, validation on Cityscapes
  - train_root='/Cityscapes/Cityspaces/'
  - val_root='/Cityscapes/Cityspaces/'
  - data_augmentation=False
  - domain_adaptation=False
2. train on GTA5, validation on GTA5
  - train_root='/GTA5/'
  - val_root='/GTA5/'
  - data_augmentation=False
  - domain_adaptation=False
3. train on GTA5, validation on GTA5 with data augmentation
  - train_root='/GTA5/'
  - val_root='/GTA5/'
  - data_augmentation=True
  - domain_adaptation=False
4. Domain Shift: GTA5 -> Cityscapes
  - train_root='/GTA5/'
  - val_root='/Cityscapes/Cityspaces/'
  - data_augmentation=False
  - domain_adaptation=False
4. Domain Shift: GTA5 -> Cityscapes with data augmentation
  - train_root='/GTA5/'
  - val_root='/Cityscapes/Cityspaces/'
  - data_augmentation=True
  - domain_adaptation=False
5. Domain Adaptation: GTA5 (source), Cityscapes (target) with training augmentation
  - train_root='/GTA5/'
  - val_root='/Cityscapes/Cityspaces/'
  - target_root='/Cityscapes/Cityscapes/'
  - data_augmentation=True
  - domain_adaptation=True
  - adv_factor=0.001
  - discr
    - 'fc' for *FCDiscriminator*
    - 'light' for *LightFCDiscriminator*
    - 'light-thin' for *LightLightFCDiscriminator*

In [22]:
class arguments():
    model = 'STDC-net'  #'BiSeNetv1'
    pretrain_path = "/content/Drive/MyDrive/Colab Notebooks/checkpoints/STDCNet813M_73.91.tar"
    use_conv_last = False
    num_epochs = 1  #50
    epoch_start_i = 0
    checkpoint_step = 1
    validation_step = 1 #5
    batch_size = 8
    learning_rate = 1e-3
    learning_rate_D = 1e-4
    optimizer = 'adam'
    optimizer_D = 'adam'
    num_classes = 19
    num_workers = 2
    save_model_path = '/content/Drive/MyDrive/Colab Notebooks/Partial models/'
    saved_model = f'cpu_Saved_model_epoch_{epoch_start_i}.pth'  # put the name of the .pth to load
    train_root = 'GTA5/'
    val_root ='Cityscapes/Cityspaces/'
    target_root = 'Cityscapes/Cityspaces/'
    data_augmentation = True
    domain_adaptation = True
    discr = 'fc'  #'fc', 'light', 'light-thin'
    adv_factor = 0.001
main_args = arguments()

main(main_args, eval_only=False)

use pretrain model /content/Drive/MyDrive/Colab Notebooks/checkpoints/STDCNet813M_73.91.tar
start train with domain adaptation
Train DA


epoch 1, lr 0.001000, lr_discriminator 0.000100:   1%|          | 16/1664 [04:48<8:06:24, 17.71s/it, loss_seg=8.129649, loss_adv=0.000673, loss_D=0.713733] 

KeyboardInterrupt: 